<h1 align="center">Deep Learning - Master in Deep Learning of UPM</h1>

En el ejercicio práctico de hoy utilizaremos como ejemplo para aplicar shallow embeddings el ya famoso _toy dataset_ "KarateClub" donde un club de Karate tras un conflicto, se separó en dos "Mr. Hi" y "Officer". Cada nodo hace referencia a un miembro y cada arista a si dos miembros han interactuado fuera del club. Nuestro objetivo será utilizar shallow embeddings para obtener representaciones de cada uno de los nodos para tratar de predecir el club de cierto miembro dada sus relaciones fuera del club con el resto de miembros.

In [225]:
import random
from typing import List, Dict, Any

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib.patches as mpatches
from IPython.display import HTML
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from tqdm.notebook import tqdm
from torch_geometric.datasets import TUDataset
from torch_geometric.utils import to_networkx

*NO TOCAR*

Esta clase nos servirá para visualizaciones tanto estáticas como dinámicas del grafo

In [226]:
class GraphVisualizer:
    def __init__(self, G: nx.Graph, cmap='Pastel1', seed=42):
        self.G = G
        self.cmap = cmap
        self.fig, self.ax = plt.subplots(figsize=(6, 6))
        self.pos = nx.spring_layout(G, seed=42)
        self.node_colors = self._generate_node_colors()
        self.seed = seed

    def _generate_node_colors(self):
        """Genera colores para los nodos basados en sus etiquetas."""
        if not hasattr(self.G, 'labels') or self.G.labels is None:
            self.label_to_color = {node: '#A1D6CB' for node in self.G.nodes}
            return ['#A1D6CB'] * len(self.G.nodes)
        else:
            # Extraemos las etiquetas de los nodos
            labels = self.G.labels.values() # Asume que el grafo tiene un atributo 'labels' con las etiquetas de los nodos

            # Creamos una lista de colores para las etiquetas
            color_map = plt.colormaps[self.cmap]  # Colores automáticos
            unique_labels, num_labels = set(labels), len(set(labels))
            self.label_to_color = {label: color_map(i / num_labels) for i, label in enumerate(unique_labels)}

            # Extraemos los colores para cada nodo
            return [self.label_to_color[node] for node in labels]
    
    def _random_walk_strategy(self, neighbors: List[int], walk: List[int] = None, **kwargs) -> int:
        selected = random.choice(neighbors) # Elegimos un vecino aleatorio
        return selected, None # Devolvemos el vecino seleccionado
    
    def _node2vec_walk_strategy(self, neighbors: List[int], walk: List[int] = None, p=1, q=1, **kwargs):
        """
        p: Return parameter - Controla la probabilidad de volver al nodo anterior, valores altos(>1) reducen la probabilidad de regresar.
        q: In-out parameter - Controla la probabilidad de visitar nodos no visitados recientemente, valores altos(>1) favorecen la exploración.
        """

        if len(walk) == 1:
            return random.choice(neighbors), None

        prev_node = walk[-2] # Nodo previo (penúltimo nodo visitado)
        weights = []

        for neighbor in neighbors:
            if neighbor == prev_node: # El vecino es el nodo previo
                weights.append(1 / p)
            elif self.G.has_edge(prev_node, neighbor): # El vecino es vecino directo del nodo previo
                weights.append(1)
            else: # El vecino es un vecino lejano
                weights.append(1 / q)

        # Normalizamos las probabilidades
        total = sum(weights)
        weights = [prob / total for prob in weights]

        # Elegimos el siguiente nodo en función de las probabilidades
        selected = random.choices(neighbors, weights=weights, k=1)[0]

        return selected, weights

    def _init_animation(self):
        """Inicializa la figura para la animación."""
        self.ax.clear()
        nx.draw(self.G, self.pos, ax=self.ax, with_labels=True, node_color=self.node_colors, font_size=8, edge_color="gray")
        return self.ax,

    def _update_animation(self, frame):
        """Actualiza la animación en cada frame."""
        self.ax.clear()
        nx.draw(self.G, self.pos, ax=self.ax, with_labels=True, node_color=self.node_colors, font_size=8, edge_color="gray")

        step = frame // 2
        current_node = self.walk[step]
        visited_nodes = self.walk[:step + 1]
        visited_edges = [(self.walk[i], self.walk[i + 1]) for i in range(len(visited_nodes) - 1)]

        # Dibujar nodos y aristas visitados
        nx.draw_networkx_nodes(self.G, self.pos, ax=self.ax, nodelist=visited_nodes, node_color="yellow")
        nx.draw_networkx_edges(self.G, self.pos, ax=self.ax, edgelist=visited_edges, edge_color="red", width=2, alpha=0.3)

        # Destacar el nodo actual
        nx.draw_networkx_nodes(self.G, self.pos, ax=self.ax, nodelist=[current_node], node_color="green")

        neighbors_info = ""
        if frame % 2 == 1: # Expansión
            neighbors = list(nx.all_neighbors(self.G, self.node))
            neighbors_edges = [(self.node, neighbor) for neighbor in neighbors]
            selected, weights = self.R(neighbors, walk=self.walk, **self.R_params)
            self.node = selected
            self.walk.append(self.node)
            nx.draw_networkx_edges(self.G, self.pos, ax=self.ax, edgelist=neighbors_edges, edge_color="blue", width=2, alpha=0.3)

            if weights:
                neighbors_weights = {neighbor: weight for neighbor, weight in zip(neighbors, weights)}
            else:
                neighbors_weights = {neighbor: 1./len(neighbors) for neighbor in neighbors}
            neighbors_info = "\n".join([f"Node {neighbor}: {weight:.2f}" for neighbor, weight in neighbors_weights.items()])

        self.ax.text(0.01, 0.99, f'Start Node: {self.start_node}\nStep: {step + 1}\nCurrent Walk: {", ".join(map(str, visited_nodes))}\n{neighbors_info}',
                     transform=self.ax.transAxes, fontsize=9, verticalalignment='top',
                     bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))

        return self.ax,

    def visualize(self, edge_labels=None, legend=True):
        # Dibujamos los nodos y las aristas
        nx.draw_networkx_nodes(self.G, ax=self.ax, pos=self.pos, node_color=self.node_colors, node_size=500)
        nx.draw_networkx_edges(self.G, ax=self.ax, pos=self.pos, alpha=0.3)
        
        # Dibujamos las etiquetas de los nodos (IDs)
        nx.draw_networkx_labels(self.G, self.pos, labels={node: node for node in self.G.nodes()}, font_size=8)

        # Dibujamos las etiquetas de las aristas
        if edge_labels:
            nx.draw_networkx_edge_labels(self.G, self.pos, edge_labels=edge_labels, font_size=8)

        # Creamos la leyenda
        if legend:
            legend_patches = [mpatches.Patch(color=color, label=label) for label, color in self.label_to_color.items()]
            self.ax.legend(handles=legend_patches, loc='best')
 
        plt.show()

    def visualize_walk(self,
                        R, # Función R o estrategia de camino 
                        start_node: int, # Nodo de inicio
                        walk_length: int, # Longitud del camino
                        save_path: str = None, # Ruta donde guardar la animación
                        **kwargs: Dict[str, Any]
                    ) -> FuncAnimation:
        """Crea y retorna la animación de un camino dada cierta estrategia."""
        match R:
            case 'random_walk':
                self.R = self._random_walk_strategy
            case 'node2vec':
                self.R = self._node2vec_walk_strategy
        self.R_params = kwargs
        self.start_node = start_node
        self.walk_length = walk_length
        self.walk = [start_node]
        self.node = start_node

        """Crea y retorna la animación del random walk."""
        ani = FuncAnimation(self.fig, self._update_animation, frames=2 * self.walk_length, init_func=self._init_animation, blit=False, interval=1000)
        
        if save_path:
            # Guarda la animación como un GIF
            ani.save(save_path, writer='pillow')

        plt.close(self.fig)
        return ani
    
    def visualize_graph_with_pca(self, features):
        embeddings = features.drop(columns=['label']) # Eliminamos la columna de etiquetas
        pca = PCA(n_components=2, random_state=42)
        coords = pca.fit_transform(embeddings)

        pos = {int(node): (coords[i, 0], coords[i, 1]) for i, node in enumerate(features.index)}
        
        # Dibujar el grafo con las coordenadas PCA
        nx.draw_networkx_nodes(self.G, pos, node_color=self.node_colors, cmap="Set2", node_size=500)
        nx.draw_networkx_edges(self.G, pos, alpha=0.3)
        
        # Dibujar las etiquetas de los nodos
        nx.draw_networkx_labels(self.G, pos, labels={node: node for node in self.G.nodes()}, font_size=8)
        
        plt.title("Visualización del grafo con PCA")
        plt.show()

# Grafos con NetworkX

In [227]:
import matplotlib.pyplot as plt
import networkx as nx

G = nx.karate_club_graph() # Cargamos el grafo de la red social de un club de karate desde el propio NetworkX

In [ ]:
for node in G.nodes(data=True):
    print(node)

El _grafo_ de networkX es un objeto de Python al cual podemos agregarle atributos.

In [ ]:
# Características de cada nodo (solo la etiqueta en este caso)
features = list(G.nodes()[0].keys())
G.num_features = len(features)
# Número de clases (en este caso, dos: 'Mr. Hi' y 'Officer', haciendo referencia al club de pertenencia)
classes = set(nx.get_node_attributes(G, 'club').values())
G.num_classes = len(classes)
# Guardemos la etiqueta de cada nodo en una lista
G.labels = {node[0]: node[1]['club'] for node in G.nodes(data=True)}

print('Propiedades del grafo')
print('==============================================================')
print(f'Dataset: {G}') # Nombre del dataset
print(f'Características: {features}') # Features de cada nodo
print(f'Clases: {classes}') # Clases posibles
print(f'Grado medio: {G.number_of_edges() / G.number_of_nodes():.2f}') # Average number of nodes in the graph
print(f'Es dirigido: {G.is_directed()}') #Is the graph an undirected graph

Visualizamos el grafo con las etiquetas de los nodos

In [ ]:
visualizer = GraphVisualizer(G)
visualizer.visualize()

# Random Walk

Random Walk realiza caminos puramente aleatorios de manera que se pueda obtener una representación no sesgada de la localización del nodo dentro del grafo.

In [ ]:
# Función R (Estrategia de camino)
def random_walk_strategy(G: nx.Graph, node: int) -> int:
    neighbors = ... # Extraemos los vecinos del nodo actual
    selected = ... # Elegimos un vecino aleatorio
    return selected # Devolvemos el vecino seleccionado

# Función de RandomWalk dado un nodo
def random_walk_from_node(G: nx.Graph, start_node: int, walk_length: int) -> List[int]:
    walk = ... # Inicializamos la lista de nodos visitados con el inicial
    node = start_node # Puntero para el nodo actual

    for _ in range(walk_length - 1):
        selected = random_walk_strategy(G, node) # Elegimos un vecino aleatorio
        ... # Añadimos el vecino a la lista de nodos visitados
        ... # Actualizamos el puntero al nodo actual
    
    return walk

start_node = 31
walk_length = 5

walk = random_walk_from_node(G, start_node=start_node, walk_length=walk_length)

print(f'Random walk: {walk}')

Visualicemos como se ve la estrategia _R_ RandomWalk

In [ ]:
start_node = 31
walk_length = 10
visualizer = GraphVisualizer(G)
ani = visualizer.visualize_walk(R='random_walk', start_node=start_node, walk_length=walk_length)
HTML(ani.to_jshtml())

Esta función random_walk() nos dará un número dado de secuencias de una longitud dada desde cada uno de los nodos

In [ ]:
def random_walk(G, walk_length=10, num_walks_per_node=10):
    walks = []

    for node in G.nodes():
        for _ in range(num_walks_per_node): # Obtenemos varios random walks por nodo
            walks.append(random_walk_from_node(G, node, walk_length=walk_length))
    
    return walks

random_walks = random_walk(G, walk_length=10, num_walks_per_node=2)
print(f'Primeras 5 random walks: {random_walks[:5]}')
print(f'Últimas 5 random walks: {random_walks[-5:]}')

# Node2Vec

Node2Vec realiza también caminos aleatorios pero los pondera en función de dos parámetros:
- p: Return parameter - Controla la probabilidad de volver al nodo anterior, valores altos(>1) reducen la probabilidad de regresar.
- q: In-out parameter - Controla la probabilidad de visitar nodos no visitados recientemente, valores altos(>1) favorecen la exploración.

In [ ]:
def node2vec_walk_strategy(G: nx.Graph, node: int, walk: List[int] = None, p=1, q=1):
    """
    p: Return parameter - Controla la probabilidad de volver al nodo anterior, valores altos(>1) reducen la probabilidad de regresar.
    q: In-out parameter - Controla la probabilidad de visitar nodos no visitados recientemente, valores altos(>1) favorecen la exploración.
    """
    neighbors = list(G.neighbors(node)) # Extraemos los vecinos del nodo actual

    if len(walk) == 1:
        selected = ... # Elegimos un vecino aleatorio
        weights = None
        return selected, weights

    prev_node = ... # Nodo previo (penúltimo nodo visitado)
    weights = []

    for neighbor in neighbors:
        if ... # El vecino es el nodo previo
            ...
        elif ... # El vecino es vecino directo del nodo previo
            ...
        else: # El vecino es un vecino lejano
            ...

    # Normalizamos las probabilidades
    total = sum(weights)
    weights = [prob / total for prob in weights]

    # Elegimos el siguiente nodo en función de las probabilidades
    selected = random.choices(neighbors, weights=weights, k=1)[0]

    return selected, weights
    

def node2vec_walk_from_node(G: nx.Graph, start_node: int, walk_length: int, p: float, q: float) -> List[int]:
    walk = [start_node] # Inicializamos la lista de nodos visitados con el inicial
    node = start_node # Puntero para el nodo actual

    for _ in range(walk_length - 1):
        selected, _ = node2vec_walk_strategy(G, node, walk=walk, p=p, q=q)
        walk.append(selected) # Añadimos el vecino a la lista de nodos visitados
        node = selected

    return walk

start_node = 31
walk_length = 10
p = 2 
q = 0.5

walk = node2vec_walk_from_node(G, start_node=start_node, walk_length=walk_length, p=p, q=q)
print(f'Node2Vec walk: {walk}')

Visualicemos como se ve la estrategia _R_ Node2Vec

In [ ]:
start_node = 11
walk_length = 10
p = 2
q = 0.5

visualizer = GraphVisualizer(G)
ani = visualizer.visualize_walk(R='node2vec', start_node=start_node, walk_length=walk_length, p=p, q=q)
HTML(ani.to_jshtml())

Esta función, al igual que random_walk(), nos dará cierto número de secuencias de cierta longitud dada pero con la estrategia _R_ propia de Node2Vec

In [ ]:
def node2vec(G, walk_length=10, num_walks_per_node=10, p=1, q=1):
    walks = []

    for node in G.nodes():
        for _ in range(num_walks_per_node): # Obtenemos varios random walks por nodo
            walks.append(node2vec_walk_from_node(G, node, walk_length=walk_length, p=p, q=q))
    
    return walks


walk_length = 10
num_walks_per_node = 2
p = 2
q = 0.5

node2vec_walks = node2vec(G, walk_length=walk_length, num_walks_per_node=num_walks_per_node, p=p, q=q)
print(f'Primeras 5 Node2Vec walks: {node2vec_walks[:5]}')
print(f'Últimas 5 Node2Vec walks: {node2vec_walks[-5:]}')

# Word2Vec para obtener representaciones de los caminos

Word2Vec nos permite obtener representaciones para ciertos elementos de un vocabulario dado. Cada representación viene dada por la aparición de una _palabra_ (o _nodo_ en este caso) en el contexto de _frases_ (o _caminos_).

El objetivo a la hora de entrenar el modelo (ya implementado en _gensim_) será generalmente predecir cierto nodo enmascarado solo teniendo acceso al resto de la secuencia. De esta forma, en el _embedding_ de cada nodo, se refleja la información estructural dados los caminos con los que entrenemos.

In [237]:
walk_length = 10
num_walks_per_node = 1
p = 2
q = 0.5

# walks = random_walk(G, walk_length=walk_length, num_walks_per_node=num_walks_per_node)
walks = node2vec(G, walk_length=walk_length, num_walks_per_node=num_walks_per_node, p=p, q=q)

Word2Vec necesita que los elementos (palabras) sean _strings_.

In [238]:
walks = [[str(i) for i in walk] for walk in walks]

In [ ]:
# Instanciamos el modelo Word2Vec
embedder = Word2Vec(
   vector_size = 100, # Tamaño del vector de embeddings
   window = 4, # Tamaño de la ventana
   sg = 1, # Usamos Skip-Gram
   hs = 0, # Usamos Negative Sampling
   negative = 10, # Número de muestras negativas
   alpha = 0.03, # Tasa de aprendizaje
   min_alpha = 0.0001, # Tasa de aprendizaje mínima
   min_count = 1, # Número mínimo de veces que una palabra debe aparecer en el corpus
   seed = 42 # Semilla para reproducibilidad
)
# Construimos el vocabulario
embedder.build_vocab(
   corpus_iterable=walks, # Corpus de random walks
   progress_per=2, # Cada cuántas iteraciones mostrar el progreso
)

# Entrenamos el modelo
embedder.train(
   corpus_iterable=walks, # Corpus de random walks
   total_examples=embedder.corpus_count, # Número total de ejemplos
   epochs=20, # Número de épocas
   report_delay=1 # Cada cuántas épocas mostrar el progreso
)

In [240]:
vocabulary = list(embedder.wv.key_to_index.keys())

assert len(vocabulary) == G.number_of_nodes(), "El tamaño del vocabulario no coincide con el número de nodos del grafo"

Esta clase será un _wrapper_ de la de Word2Vec de _gensim_ que simplificará y adaptará el uso de esta a nuestra tarea.

In [241]:
class NodeEncoder:
    def __init__(self, *args, **kwargs):
        self.encoder = Word2Vec(*args, **kwargs)
    
    def fit(self, walks, epochs: int=20, progress_per: int=2):
        if not isinstance(walks[0][0], str):
            walks = [[str(i) for i in walk] for walk in walks]

        self.encoder.build_vocab(walks, progress_per=progress_per)
        self.encoder.train(walks, total_examples=self.encoder.corpus_count, epochs=epochs, report_delay=1)

    def transform(self, X):
        if not isinstance(X[0], str):
            X = [str(i) for i in X]
        
        return {i: self.encoder.wv[i] for i in X if i in self.encoder.wv}

    def fit_transform(self, walks, epochs=20, progress_per=2):
        self.fit(walks, epochs=epochs, progress_per=progress_per)
        X = list(self.encoder.wv.key_to_index.keys())
        return self.transform(X)

In [ ]:
encoder = NodeEncoder(
    vector_size = 10, # Tamaño del vector de embeddings
    window = 4, # Tamaño de la ventana
    sg = 1, # Usamos Skip-Gram
    hs = 0, # Usamos Negative Sampling
    negative = 10, # Número de muestras negativas
    alpha = 0.03, # Tasa de aprendizaje
    min_alpha = 0.0001, # Tasa de aprendizaje mínima
    min_count = 1, # Número mínimo de veces que una palabra debe aparecer en el corpus
    seed = 42 # Semilla para reproducibilidad
)

embeddings = encoder.fit_transform(walks, epochs=20, progress_per=2) # Generamos los embeddings

embeddings = pd.DataFrame(embeddings).T # Mostramos los primeros embeddings en forma de DataFrame
embeddings.head()

Mapearemos las etiquetas de cada nodo a un valor entero para poder entrenar un clasificador

In [ ]:
str2label = {'Mr. Hi': 0, 'Officer': 1} # Mapeo de etiquetas a enteros
labels = {str(idx): [str2label[label]] for idx, label in G.labels.items()} # Obtenemos las etiquetas de los nodos

labels = pd.DataFrame(labels).T # Mostramos las etiquetas en forma de DataFrame
labels.columns = ['label']

labels.head()

Finalmente, combinaremos los _embeddings_ (o _features_) y las etiquetas teniendo como resultado un caso de "clasificación de datos tabulares".

In [ ]:
df = pd.merge(embeddings, labels, left_index=True, right_index=True) # Unimos los embeddings y las etiquetas

df.head()

In [245]:
def generate_dataset(df, eval_size=0.2, random_state=42):
    X, y = df.iloc[:, :-1], df.iloc[:, -1] # Separamos las características y las etiquetas
    X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=eval_size, random_state=random_state)

    return X_train, X_eval, y_train, y_eval

def train_classifier(X_train, y_train, X_eval=None, y_eval=None, classifier=None):
    if classifier is None:
        classifier = RandomForestClassifier() # Inicializamos el clasificador
    classifier = classifier.fit(X_train, y_train) # Entrenamos el clasificador

    if X_eval is not None and y_eval is not None:
        preds = classifier.predict(X_eval) # Predecimos las etiquetas de entrenamiento
        acc = accuracy_score(y_eval, preds)
        f1 = f1_score(y_eval, preds)
        print(f'Accuracy: {acc:.2f} - F1 Score: {f1:.2f}')

    return classifier

In [ ]:
X_train, X_eval, y_train, y_eval = generate_dataset(df, eval_size=0.2) # Generamos el dataset
classifier = RandomForestClassifier(
    n_estimators=100, # Número de árboles
    max_depth=5, # Profundidad máxima
    random_state=42 # Semilla para reproducibilidad
)

classifier = train_classifier(X_train, y_train, X_eval, y_eval) # Entrenamos el clasificador

El objetivo de experimentación será obtener un rendimiento decente (f1 y acc > 0.85) con la única restricción de que el tamaño de camino máximo que podemos usar es de 5.

In [ ]:
### HIPERPARÁMETROS ###

# Estrategia de camino
walk_length = 5
num_walks_per_node = ...
p = ...
q = ...

# Encoder
vector_size = ... # Tamaño del vector de embeddings
window = 4 # Tamaño de la ventana
sg = 1 # Usamos Skip-Gram
hs = 0 # Usamos Negative Sampling
negative = 10 # Número de muestras negativas
alpha = 0.03 # Tasa de aprendizaje
min_alpha = 0.0001 # Tasa de aprendizaje mínima
min_count = 1 # Número mínimo de veces que una palabra debe aparecer en el corpus
seed = 42 # Semilla para reproducibilidad

epochs = 20
progress_per = 2

# Clasificador
eval_size = 0.6
classifier = RandomForestClassifier(
    n_estimators=..., # Número de árboles
    max_depth=..., # Profundidad máxima
    random_state=seed # Semilla para reproducibilidad
)

# Obtenemos los caminos
# walks = random_walk(G, walk_length=walk_length, num_walks_per_node=num_walks_per_node)
walks = node2vec(G, walk_length=walk_length, num_walks_per_node=num_walks_per_node, p=p, q=q)

encoder = NodeEncoder(
    vector_size = vector_size, # Tamaño del vector de embeddings
    window = window, # Tamaño de la ventana
    sg = sg, # Usamos Skip-Gram
    hs = hs, # Usamos Negative Sampling
    negative = negative, # Número de muestras negativas
    alpha = alpha, # Tasa de aprendizaje
    min_alpha = min_alpha, # Tasa de aprendizaje mínima
    min_count = min_count, # Número mínimo de veces que una palabra debe aparecer en el corpus
    seed = seed # Semilla para reproducibilidad
)

# Codificamos los nodos
embeddings = encoder.fit_transform(walks, epochs=epochs, progress_per=progress_per) # Generamos los embeddings
embeddings = pd.DataFrame(embeddings).T # Mostramos los primeros embeddings en forma de DataFrame

str2label = {'Mr. Hi': 0, 'Officer': 1} # Mapeo de etiquetas a enteros
labels = {str(idx): [str2label[label]] for idx, label in G.labels.items()} # Obtenemos las etiquetas de los nodos

labels = pd.DataFrame(labels).T # Mostramos las etiquetas en forma de DataFrame
labels.columns = ['label']

df = pd.merge(embeddings, labels, left_index=True, right_index=True) # Unimos los embeddings y las etiquetas

X_train, X_eval, y_train, y_eval = generate_dataset(df, eval_size=eval_size) # Generamos el dataset
classifier = train_classifier(X_train, y_train, X_eval, y_eval, classifier=classifier) # Entrenamos el clasificador

La siguiente función reduce la dimensión obtenida de los embeddings mediante PCA a 2 dimensiones de manera que se pueden entender como coordenadas (x, y) y ser ploteadas en función de esto.

In [ ]:
visualizer = GraphVisualizer(G)
visualizer.visualize_graph_with_pca(features=df)

# Embeddings para aristas

Para la predicción de aristas vamos a tomar el mismo dataset y transformarlo de manera que la etiqueta de la arista que conecta dos nodos será:
- 0: si los dos miembros eran de clubs diferentes
- 1: si los dos miembros pertenecían al mismo club

In [249]:
# Obtener las etiquetas de los nodos (clubes)
node_clubs = nx.get_node_attributes(G, "club")

# Etiquetar las aristas: 1 si los nodos pertenecen al mismo club, 0 si no
edge_labels = {}
for u, v in G.edges():
    edge_labels[(u, v)] = 1 if node_clubs[u] == node_clubs[v] else 0

Visualicemos como están etiquetadas las aristas en función de nuestro criterio

In [ ]:
visualizer = GraphVisualizer(G)
visualizer.visualize(edge_labels=edge_labels)

Obtengamos de la misma manera los embeddings para los nodos

In [251]:
### HIPERPARÁMETROS ###

# Estrategia de camino
walk_length = 5
num_walks_per_node = 5
p = 1
q = 1

# Encoder
vector_size = 10 # Tamaño del vector de embeddings
window = 4 # Tamaño de la ventana
sg = 1 # Usamos Skip-Gram
hs = 0 # Usamos Negative Sampling
negative = 10 # Número de muestras negativas
alpha = 0.03 # Tasa de aprendizaje
min_alpha = 0.0001 # Tasa de aprendizaje mínima
min_count = 1 # Número mínimo de veces que una palabra debe aparecer en el corpus
seed = 42 # Semilla para reproducibilidad

epochs = 20
progress_per = 2

# Obtenemos los caminos
# walks = random_walk(G, walk_length=walk_length, num_walks_per_node=num_walks_per_node)
walks = node2vec(G, walk_length=walk_length, num_walks_per_node=num_walks_per_node, p=p, q=q)

encoder = NodeEncoder(
    vector_size = vector_size, # Tamaño del vector de embeddings
    window = window, # Tamaño de la ventana
    sg = sg, # Usamos Skip-Gram
    hs = hs, # Usamos Negative Sampling
    negative = negative, # Número de muestras negativas
    alpha = alpha, # Tasa de aprendizaje
    min_alpha = min_alpha, # Tasa de aprendizaje mínima
    min_count = min_count, # Número mínimo de veces que una palabra debe aparecer en el corpus
    seed = seed # Semilla para reproducibilidad
)

# Codificamos los nodos
embeddings = encoder.fit_transform(walks, epochs=epochs, progress_per=progress_per) # Generamos los embeddings

Esta función obtendrá la representación para cada una de las aristas aplicando una operación sobre los embeddings de los nodos conectados.

In [ ]:
def edge_embedding(G, node_embeddings, op='mean'):
    edge_embeddings = {}
    
    for u, v in G.edges():
        u_embedding = node_embeddings.get(str(u), None)
        v_embedding = node_embeddings.get(str(v), None)
        
        if u_embedding is not None and v_embedding is not None:
            match op:
                case 'mean':
                    edge_embedding = (u_embedding + v_embedding) / 2
                case 'hadamard':
                    edge_embedding = u_embedding * v_embedding
                case 'sum':
                    edge_embedding = u_embedding + v_embedding
                case 'sub':
                    edge_embedding = u_embedding - v_embedding
                case 'concat':
                    edge_embedding = np.concatenate([u_embedding, v_embedding])
                case _:
                    raise ValueError(f'Operación no soportada: {op}')
            
            edge_embeddings[f"{(u, v)}"] = edge_embedding
    
    return edge_embeddings

edge_embeddings = edge_embedding(G, node_embeddings=embeddings, op='sum')

pd.DataFrame(edge_embeddings).T 

Combinamos las representaciones de las aristas y la etiqueta en un mismo DataFrame

In [ ]:
df = pd.merge(pd.DataFrame(edge_embeddings).T, pd.Series({f"{k}": v for k, v in edge_labels.items()}, name='label'), left_index=True, right_index=True)

df.head()

De la misma manera, hacemos splits y entrenamos como si se tratara de clasificación de datos tabulares.

In [ ]:
X_train, X_eval, y_train, y_eval = generate_dataset(df, eval_size=0.2) # Generamos el dataset
classifier = RandomForestClassifier(
    n_estimators=100, # Número de árboles
    max_depth=5, # Profundidad máxima
    random_state=42 # Semilla para reproducibilidad
)

classifier = train_classifier(X_train, y_train, X_eval, y_eval) # Entrenamos el clasificador

### Line graph

Alternativamente podemos crear un grafo en el que cada nodo represente una arista del grafo original. Dos nodos estarán conectados si entre las aristas compartían un nodo en común.

In [255]:
line_G = nx.line_graph(G)
line_G.labels = edge_labels # Añadimos las etiquetas de las aristas con el anterior criterio

In [ ]:
visualizer = GraphVisualizer(G)
visualizer.visualize(edge_labels=edge_labels)

El grafo resultante _lineG_ tendrá tantos nodos como el original aristas.

In [ ]:
line_G_visualizer = GraphVisualizer(line_G)
line_G_visualizer.visualize()

Aplicamos nuestra pipeline para node encoding

In [258]:
### HIPERPARÁMETROS ###

# Estrategia de camino
walk_length = 5
num_walks_per_node = 5
p = 1
q = 1

# Encoder
vector_size = 10 # Tamaño del vector de embeddings
window = 4 # Tamaño de la ventana
sg = 1 # Usamos Skip-Gram
hs = 0 # Usamos Negative Sampling
negative = 10 # Número de muestras negativas
alpha = 0.03 # Tasa de aprendizaje
min_alpha = 0.0001 # Tasa de aprendizaje mínima
min_count = 1 # Número mínimo de veces que una palabra debe aparecer en el corpus
seed = 42 # Semilla para reproducibilidad

epochs = 20
progress_per = 2

# Obtenemos los caminos
# walks = random_walk(line_G, walk_length=walk_length, num_walks_per_node=num_walks_per_node)
walks = node2vec(line_G, walk_length=walk_length, num_walks_per_node=num_walks_per_node, p=p, q=q)

encoder = NodeEncoder(
    vector_size = vector_size, # Tamaño del vector de embeddings
    window = window, # Tamaño de la ventana
    sg = sg, # Usamos Skip-Gram
    hs = hs, # Usamos Negative Sampling
    negative = negative, # Número de muestras negativas
    alpha = alpha, # Tasa de aprendizaje
    min_alpha = min_alpha, # Tasa de aprendizaje mínima
    min_count = min_count, # Número mínimo de veces que una palabra debe aparecer en el corpus
    seed = seed # Semilla para reproducibilidad
)

# Codificamos los nodos
embeddings = encoder.fit_transform(walks, epochs=epochs, progress_per=progress_per) # Generamos los embeddings

In [ ]:
df = pd.merge(pd.DataFrame(edge_embeddings).T, pd.Series({f"{k}": v for k, v in edge_labels.items()}, name='label'), left_index=True, right_index=True)

df.head()

In [ ]:
X_train, X_eval, y_train, y_eval = generate_dataset(df, eval_size=0.2) # Generamos el dataset
classifier = RandomForestClassifier(
    n_estimators=100, # Número de árboles
    max_depth=5, # Profundidad máxima
    random_state=42 # Semilla para reproducibilidad
)

classifier = train_classifier(X_train, y_train, X_eval, y_eval) # Entrenamos el clasificador

# Embeddings para grafos

En este ejercicio práctico vamos a trabajar con el dataset **MUTAG**.

Se trata de un dataset de clasificación de grafos. Cada grafo representa una molécula, donde los nodos corresponden a átomos y aristas representan enlaces químicos. Se utiliza para predecir si una molécula tiene propiedades mutagénicas o no. Dispone de 188 grafos y es un estándar para evaluar modelos en tareas de clasificación de grafos.

In [261]:
def load_mutag(n=None, centinel_node=False):
    dataset = TUDataset(root='data/TUDataset', name='MUTAG') # Cargar el dataset MUTAG
    dataset = dataset[:n] if n else dataset # Limitar el número de grafos si es necesario
    
    # Crear un grafo de NetworkX vacío
    combined_graph = nx.Graph()
    graphs_info = {}

    # Iterar sobre todos los grafos del dataset y combinarlos
    for graph_idx, graph in enumerate(dataset):
        # Convertir cada grafo de PyTorch Geometric a NetworkX
        nx_graph = to_networkx(graph, to_undirected=True)

        # Renombrar los nodos para evitar colisiones
        nx_graph = nx.relabel_nodes(nx_graph, lambda x: x + len(combined_graph))

        # Añadir un nodo centinela al grafo
        if centinel_node:
            nx_graph.add_node(f'c_{graph_idx}')
            for node in nx_graph.nodes:
                nx_graph.add_edge(f'c_{graph_idx}', node)

        # Les añadimos a cada nodo el grafo de pertenencia como atributo
        nx.set_node_attributes(nx_graph, {node: graph_idx for node in nx_graph.nodes}, 'graph_id')
        
        graphs_info[graph_idx] = {
            'node_ids': list(nx_graph.nodes),
            'label': graph.y.numpy().item(),
        }

        # Agregar al grafo combinado
        combined_graph = nx.compose(combined_graph, nx_graph)
    
    return combined_graph, graphs_info

Veamos como se ven los n primeros grafos

*NOTA*: Por como se visualizan con networkX puede parecer que a veces hay dos grafos que están conexos, pero no lo están.

In [ ]:
n = 3

graph, graphs_info = load_mutag(n=n)
visualizer = GraphVisualizer(graph)
visualizer.visualize(legend=False)

In [ ]:
graphs_info # Información de cada sub-grafo

Veamos como se recorren con random walk o node2vec

In [ ]:
R = 'node2vec'
start_node = 17
walk_length = 10
p = 2
q = 0.5

visualizer = GraphVisualizer(graph)
ani = visualizer.visualize_walk(R=R, start_node=start_node, walk_length=walk_length, p=p, q=q)
HTML(ani.to_jshtml())

## Enfoque trivial por agregación

Computaremos los embeddings de todos los nodos de cada sub-grafo y los agregaremos según cierta operación

Ahora carguémoslo entero

*IMPORTANTE*: si la persona leyendo esto lo está ejecutando en local y tiene cierto cariño a su ordenador, recomiendo no tratar de visualizar el grafo entero :)

In [ ]:
graph, graphs_info = load_mutag() # Cargamos el dataset MUTAG entero

# Información sobre el grafo combinado
print(f"Número total de nodos: {graph.number_of_nodes()}")
print(f"Número total de aristas: {graph.number_of_edges()}")

Computamos node2vec sobre el grafo completo (todos los sub-grafos inconexos)

Agreguemos los embeddings de cada sub-grafo. Para ello se requiere crear una función _compute\_graph\_embedding()_ que recibirá el parámetro _op_ que podrá recibir tres valores:

- 'mean': realizará la media
- 'sum': realizará la suma
- 'hadamard': realizará la multiplicación de Hadamard

Extraemos los embeddings de cada grafo

Extraemos las etiquetas

Combinamos los embeddings y las etiquetas

A entrenar!

## Agregando un Nodo Centinela

Recordemos que un nodo centinela es un nodo _virtual_ que se conecta a todos los nodos de cada sub-grafo. Usaremos el embedding de ese nodo como una representación de todo el sub-grafo.

Con un simple parámetro en la función _load_mutag()_ se añade un nodo centinela a cada sub-grafo con el id "c_{graph_idx}"

In [ ]:
graph, graphs_info = load_mutag(centinel_node=True) # Cargamos el dataset MUTAG entero con un nodo centinela en cada subgrafo

graphs_info[0]

Y aquí no hay nada que agregar, directamente computamos node2vec.

Extraemos los embeddings de cada centinela como las _features_

Y extraemos las etiquetas

Combinamos embeddings y etiquetas

A entrenar!

---------------------------